**IMPORT PYNGROK TO RUN STREAMLIT ON GOOGLE COLAB**

In [2]:
!pip install streamlit pyngrok

**CREATING APP.PY FILE TO RUN STREAMLIT IN LOCAL SERVER**

In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

# --- Page Configuration ---
st.set_page_config(page_title="Rossmann Sales Dashboard", layout="wide")

# --- Header ---
st.markdown("""
    <h1 style='text-align: center; color: #FAFAFA; font-size: 40px;'>
        📊 Rossmann Interactive Sales Dashboard
    </h1>
""", unsafe_allow_html=True)

# --- File Upload ---
uploaded_file = st.file_uploader("Upload Rossmann Sales Data", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file, parse_dates=["Date"])
else:
    st.warning("Please upload the Rossmann dataset to proceed.")
    st.stop()

# --- Sidebar Filters ---
with st.sidebar:
    st.header("🔍 Filter Options")

    store_ids = st.multiselect("Select Store(s)", options=sorted(df["Store"].unique()))
    store_types = st.multiselect("Select Store Type(s)", options=sorted(df["StoreType"].unique()))
    assortments = st.multiselect("Select Assortment(s)", options=sorted(df["Assortment"].unique()))
    date_range = st.date_input("Select Date Range",
                               [df["Date"].min().date(), df["Date"].max().date()])

# --- Apply Filters ---
filtered_df = df.copy()

# Date filter
if len(date_range) == 2:
    start_date = pd.to_datetime(date_range[0])
    end_date = pd.to_datetime(date_range[1])
    filtered_df = filtered_df[(filtered_df["Date"] >= start_date) & (filtered_df["Date"] <= end_date)]

# Store filter
if store_ids:
    filtered_df = filtered_df[filtered_df["Store"].isin(store_ids)]

# StoreType filter
if store_types:
    filtered_df = filtered_df[filtered_df["StoreType"].isin(store_types)]

# Assortment filter
if assortments:
    filtered_df = filtered_df[filtered_df["Assortment"].isin(assortments)]

# --- KPIs ---
total_sales = filtered_df["Sales"].sum()
avg_customers = filtered_df["Customers"].mean()
promo_sales = filtered_df[filtered_df["Promo"] == 1]["Sales"].sum()

col1, col2, col3 = st.columns(3)
col1.metric("Total Sales", f"€{total_sales:,}")
col2.metric("Average Customers", f"{avg_customers:,.0f}")
col3.metric("Promo Sales", f"€{promo_sales:,}")

# --- Monthly Sales Trend ---
st.markdown("### 📈 Monthly Sales Trend")
filtered_df["Month"] = filtered_df["Date"].dt.to_period("M")
monthly_sales = filtered_df.groupby(filtered_df["Month"].astype(str))["Sales"].sum().reset_index()
fig_time = px.line(monthly_sales, x="Month", y="Sales", markers=True, template="plotly_dark")
st.plotly_chart(fig_time, use_container_width=True)

# --- Pie Charts ---
col4, col5 = st.columns(2)
with col4:
    st.markdown("### 🏪 Sales by Store Type")
    storetype_sales = filtered_df.groupby("StoreType")["Sales"].sum().reset_index()
    fig_storetype = px.pie(storetype_sales, names="StoreType", values="Sales", hole=0.5, template="seaborn")
    st.plotly_chart(fig_storetype, use_container_width=True)

with col5:
    st.markdown("### 🧺 Sales by Assortment")
    assortment_sales = filtered_df.groupby("Assortment")["Sales"].sum().reset_index()
    fig_assortment = px.pie(assortment_sales, names="Assortment", values="Sales", hole=0.5, template="seaborn")
    st.plotly_chart(fig_assortment, use_container_width=True)

# --- Sunburst Chart ---
st.markdown("### 🌞 Sales Distribution by StoreType → Assortment → Promo")
fig_sunburst = px.sunburst(
    filtered_df,
    path=["StoreType", "Assortment", "Promo"],
    values="Sales",
    color="Promo",
    color_continuous_scale="RdBu",
    template="plotly_dark"
)
st.plotly_chart(fig_sunburst, use_container_width=True)

# --- Customers vs Sales Scatter Plot ---
st.markdown("### 📌 Customers vs Sales")
fig_scatter = px.scatter(
    filtered_df,
    x="Customers", y="Sales",
    color="StoreType",
    size="Sales",
    hover_data=["Store", "Date"],
    template="plotly_dark"
)
st.plotly_chart(fig_scatter, use_container_width=True)

# --- Segment Sales ---
chart1, chart2 = st.columns(2)
with chart1:
    st.subheader("🟢 Open vs Closed Sales")
    open_df = filtered_df.groupby("Open")["Sales"].sum().reset_index()
    open_df["Open"] = open_df["Open"].map({1: "Open", 0: "Closed"})
    fig_open = px.pie(open_df, names="Open", values="Sales", hole=0.4, template="plotly_dark")
    st.plotly_chart(fig_open, use_container_width=True)

with chart2:
    st.subheader("🔺 Promo vs No Promo Sales")
    promo_df = filtered_df.groupby("Promo")["Sales"].sum().reset_index()
    promo_df["Promo"] = promo_df["Promo"].map({1: "Promo", 0: "No Promo"})
    fig_promo = px.pie(promo_df, names="Promo", values="Sales", hole=0.4, template="plotly_dark")
    st.plotly_chart(fig_promo, use_container_width=True)

# --- Correlation Heatmap ---
st.subheader("📊 Correlation Heatmap")
corr_data = filtered_df[["Sales", "Customers", "Promo", "SchoolHoliday"]].corr()
fig_corr, ax = plt.subplots(figsize=(8, 5))
sns.heatmap(corr_data, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=0.5, ax=ax)
st.pyplot(fig_corr)

# --- Sample Data & Export ---
st.subheader("🗂 Sample Data Preview")
with st.expander("View Sample Data"):
    st.write(filtered_df.head().style.background_gradient(cmap="YlGnBu"))
    csv = filtered_df.to_csv(index=False).encode("utf-8")
    st.download_button("Download CSV", data=csv, file_name="Rossmann_Filtered_Data.csv", mime="text/csv")


Writing app.py


**Getting authenticatae token for ngrok and define a funtion to make UI interface in local server **

In [4]:
import threading
import time
from pyngrok import ngrok
ngrok.set_auth_token("2ySL0saJYSgzKIcg1NNACGs2gXP_4SCQYsLJSQ6d5Edt8M3v6")

def run():
    !streamlit run app.py &

# Start Streamlit in a separate thread
thread = threading.Thread(target=run)
thread.start()

# Wait a little for the server to start
time.sleep(3)

# Get public URL
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.172.233:8501

Streamlit app is live at: NgrokTunnel: "https://851b-34-16-172-233.ngrok-free.app" -> "http://localhost:8501"
